In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new')
# Load pre-trained model
model = BertForMaskedLM.from_pretrained('/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new')
# model.eval()

OSError: Can't load tokenizer for '/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new'. Make sure that:

- '/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new' is a correct model identifier listed on 'https://huggingface.co/models'

- or '/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new' is the correct path to a directory containing relevant tokenizer files



In [2]:
query = '宿泊代金の支払方法について教えてください。'

In [8]:
tokenized_text = ['宿泊代金', '支払方法', '教える']

In [3]:
tokenized_text = tokenizer.tokenize(query)
tokenized_text

['宿泊', '代金', 'の', '支払', '方法', 'について', '教え', 'て', 'ください', '。']

In [9]:
# ['テレビ', 'で', 'サッカー', 'の', '試合', 'を', '見る', '。']

# Mask a token that we will try to predict back with `BertForMaskedLM`
# masked_index = question.index(word)
masked_index = 1
tokenized_text[masked_index] = '[MASK]'
# ['テレビ', 'で', '[MASK]', 'の', '試合', 'を', '見る', '。']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# [571, 12, 4, 5, 608, 11, 2867, 8]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
# tensor([[ 571,   12,    4,    5,  608,   11, 2867,    8]])

In [13]:
x = outputs[0][0, masked_index].topk(10)

In [15]:
for i, index_t in enumerate(x.indices):
    index = index_t.item()
    print(tokenizer.convert_ids_to_tokens([index]))index

['教え']
['教える']
['学び']
['を']
['学ぶ']
['生きる']
['芸能']
['教育']
['学習']
['も']


In [7]:
# Predict
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0][0, masked_index].topk(10)  # 予測結果の上位5件を抽出

# Show results
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

0 1
1 野球
2 3
3 2
4 試合
5 スポーツ
6 10
7 18
8 基本
9 全


## Trying Next seq approach

In [15]:
import torch

In [3]:

from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

In [4]:
# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new')
# Load pre-trained model
model = BertForMaskedLM.from_pretrained('/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new')


Some weights of the model checkpoint at /home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
test = '朝食に[MASK]を焼いて食べました。'

In [10]:
token_ids = tokenizer.encode(test, add_special_tokens=True)

In [11]:
token_ids

[2, 2821, 6521, 893, 4, 932, 24183, 888, 12409, 13276, 881, 829, 3]

In [12]:
tokens = tokenizer.convert_ids_to_tokens(token_ids)

In [13]:
tokens

['[CLS]',
 '朝',
 '##食',
 'に',
 '[MASK]',
 'を',
 '焼い',
 'て',
 '食べ',
 'まし',
 'た',
 '。',
 '[SEP]']

In [16]:
token_ids = torch.tensor([token_ids])

In [17]:
token_ids

tensor([[    2,  2821,  6521,   893,     4,   932, 24183,   888, 12409, 13276,
           881,   829,     3]])

In [18]:
masked_index = torch.where(token_ids == tokenizer.mask_token_id)[1].tolist()[0]
print(masked_index)

4


In [19]:
result = model(token_ids)
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = token_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))

[CLS] 朝食 に パン を 焼い て 食べ まし た 。 [SEP]
[CLS] 朝食 に うどん を 焼い て 食べ まし た 。 [SEP]
[CLS] 朝食 に 肉 を 焼い て 食べ まし た 。 [SEP]
[CLS] 朝食 に 魚 を 焼い て 食べ まし た 。 [SEP]
[CLS] 朝食 に ジャガイモ を 焼い て 食べ まし た 。 [SEP]


### The following code is for BERT download for masking

In [1]:
from transformers import BertJapaneseTokenizer, BertForMaskedLM
# from sentence_transformers import SentenceTransformer, models


DEFAULT_PRIORITY = 1


def load_bert(model_path):
    tok = BertJapaneseTokenizer.from_pretrained(model_path)
    masked_model = BertForMaskedLM.from_pretrained(model_path)
    return {"tokenizer": tok, 'model': masked_model}

In [3]:
model_path = 'cl-tohoku/bert-large-japanese'

In [5]:
model = load_bert(model_path)

Downloading:   0%|          | 0.00/518 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-large-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
model.save_pretrained('/home/iftekhar/BERTS/BERT_masking/')

In [21]:
tokenizer.save_pretrained('/home/iftekhar/BERTS/BERT_masking/')

('/home/iftekhar/BERTS/BERT_masking/tokenizer_config.json',
 '/home/iftekhar/BERTS/BERT_masking/special_tokens_map.json',
 '/home/iftekhar/BERTS/BERT_masking/vocab.txt',
 '/home/iftekhar/BERTS/BERT_masking/added_tokens.json')

In [8]:
tokenizer = model['tokenizer']
model = model['model']